In [30]:
# from hmmlearn import hmm
import numpy as np
import pandas as pd
from collections import defaultdict
import datetime
import math
# import pykovy-master/ 
from pykovy.src.pykovy import chain
import matplotlib.pyplot as plt

In [19]:
df = pd.read_csv('data_sample/ap.csv')
df = df[['Client Username', 'Association Time', 'Map Location', 'Session Duration']]

In [20]:
# re-code userid
df['Client Username'] = pd.Categorical(df['Client Username'])
df['Client Username'] = df['Client Username'].cat.codes
# df = df.drop(['Client Username'], axis=1)

# recode map location
map_dict = {}
count = 0
for i in df['Map Location'].unique():
    map_dict[i] = count
    count += 1
df['Map Location'] = df['Map Location'].map(map_dict)


In [21]:
df

,Client Username,Association Time,Map Location,Session Duration
0,12,Sat Feb 24 00:38:41 PST 2018,0,7 hrs 25 min 57 sec
1,12,Sat Feb 24 08:04:39 PST 2018,0,8 hrs 10 min 25 sec
2,79,Sat Feb 24 13:16:49 PST 2018,1,2 hrs 58 min 15 sec
3,81,Fri Feb 23 16:38:44 PST 2018,1,1 hrs 48 min 10 sec
4,81,Fri Feb 23 18:26:55 PST 2018,1,10 min 26 sec
5,81,Sat Feb 24 13:42:56 PST 2018,1,2 hrs 32 min 9 sec
6,50,Sat Feb 24 11:00:26 PST 2018,2,15 min 36 sec
7,50,Sat Feb 24 11:16:03 PST 2018,2,31 min 8 sec
8,50,Sat Feb 24 11:47:11 PST 2018,2,10 min 23 sec
9,50,Sat Feb 24 11:57:34 PST 2018,2,1 hrs 13 min 30 sec


In [48]:
result = {}
result["client_username"] = [-1] * 3
result["client_username"][2] = 1
print(result)
print(24 % 24)

{'client_username': [-1, -1, 1]}
0


In [36]:
#from datetime import datetime
def convertMonth(month):
    if month == 'Jan': return 1
    elif month == 'Feb': return 2
    elif month == 'Mar': return 3
    elif month == 'Apr': return 4
    elif month == 'May': return 5
    elif month == 'Jun': return 6
    elif month == 'Jul': return 7
    elif month == 'Aug': return 8
    elif month == 'Sep': return 9
    elif month == 'Oct': return 10
    elif month == 'Nov': return 11
    elif month == 'Dec': return 12
    else: print('error input is not correct')
        
def convert_to_timeslot(duration):
    temp = duration.split()
    if len(temp) == 2: sess_duration = int(temp[0])
    elif len(temp) == 4: sess_duration = int(temp[0]) * 60 + int(temp[2])
    elif len(temp) == 6: sess_duration = (int(temp[0]) * 60 + int(temp[2])) * 60 + int(temp[4])
    else: print('error')
    return datetime.timedelta(seconds=sess_duration)

def convert_duration_to_seconds(duration):
    temp = duration.split()
    if len(temp) == 2: sess_duration = int(temp[0])
    elif len(temp) == 4: sess_duration = int(temp[0]) * 60 + int(temp[2])
    elif len(temp) == 6: sess_duration = (int(temp[0]) * 60 + int(temp[2])) * 60 + int(temp[4])
    else: print('error')
    return int(sess_duration)

def convert_to_datetime(time):
    temp = time.split()
    d_time = datetime.datetime(year=int(temp[5]), month=convertMonth(temp[1]), day=int(temp[2]), hour=int(temp[3].split(':')[0]), minute=int(temp[3].split(':')[1]), second=int(temp[3].split(':')[2]))
    return d_time

print(convert_to_datetime('Sat Feb 24 13:42:56 PST 2018'))
print(convert_duration_to_seconds('7 hrs 25 min 57 sec'))

2018-02-24 13:42:56
26757


In [29]:
#from datetime import datetime
# convert a time format
# Sat Feb 24 13:42:56 PST 2018
# to timestamp value
def convert_time_to_timestamp(input_time):
    datetime_format = convert_to_datetime(input_time)
    dt_obj = datetime.datetime.strptime(str(datetime_format),
                                        '%Y-%m-%d %H:%M:%S')
    return int(dt_obj.timestamp())
print(convert_time_to_timestamp('Sat Feb 24 13:42:56 PST 2018'))

1519508576


In [32]:
def convert_timestamp_to_index(input_ts, start_ts, window_in_minutes):
    index = math.floor((input_ts - start_ts) / (window_in_minutes * 60))
    return index
print(convert_timestamp_to_index(1519508576, 1519462800, 60))

12


In [56]:
# create a time slot 2d array each row is a user and each cols is a timeslot.
# for a particular user if there is no information on a particular time slot append(-1)
# this function is not finished
# Input:
# - df: dataframe
# - userdict: list of user id
# - start_timestamp: start time timestamp
# - end_timestamp: end time timestamp
# - window_in_minute: time window, use 60 (1 hour) first for simplicity
# 
# Output:
# - result: a hashmap (dictionary) where each map item is a 1D array where array 
#           indexes are indexed timeslot and values are "Map Location" (integer value)
#           -1 if there's no information for that timeslot
def create_time_slot_data(df, user_dict, start_timestamp, end_timestamp, window_in_minutes):
    # calculate number of index in each array
    # Formula:
    # number_of_time_entry = (diff of timestamp in second) / (number of seconds in that time window)
    number_of_time_entry = int((end_timestamp - start_timestamp) / (window_in_minutes * 60))
    
    # Addumption: start date at 0:0:0, end date at 23:59:59
    # 86400 = seconds in a day
    number_of_days = int((end_timestamp - start_timestamp) / 86400)
    
    number_of_time_entry_per_day = int(86400 / (window_in_minutes * 60))
    print(number_of_time_entry_per_day)
    result = {}
    for i in range(df.shape[0]):
        client_username = df.iloc[i,0]
        association_time = df.iloc[i,1]
        map_location = df.iloc[i,2]
        session_duration = df.iloc[i,3]
        
        # 1/ If there's no entry for this user in result dict
        #    create a new 1d array with all -1 
        #    then change values base on association time and session duration
        # 2/ If user already in the result dict
        #    then change values base on association time and session duration
        # Caveats: there's maybe duplication, ignore it for now
        if client_username not in result:
            #time_entry_ts = start_timestamp
            #while time_entry_ts < end_timestamp:
            result[client_username] = [-1] * number_of_time_entry
                #time_entry_ts += 86400
        # convert association time to timestamp
        # and calculate the total sesstion in timestamp
        association_time_ts = convert_time_to_timestamp(association_time)
        end_association_time_ts = association_time_ts + convert_duration_to_seconds(session_duration)
        if (association_time_ts > end_timestamp): 
            continue
        # TODO:
        # Need to handle the case where association timestamp < start_timestamp
        # in that case, some array entry might change
        start_index = convert_timestamp_to_index(association_time_ts, start_timestamp, window_in_minutes)
        end_index = convert_timestamp_to_index(end_association_time_ts, start_timestamp, window_in_minutes)
        # mark all the index from start index to end index to map location
        for i in range(start_index, end_index):
            result[client_username][i] = map_location
       
    print(result)
    # Now we have a result dictionary without date
    # Do some more calculation to convert this to a dict with
    # {userid, date, list of timeslot}
    output = {}
    for user in result:
        output[user] = {}
        date_ts = start_timestamp
        output[user][date_ts] = [-1] * number_of_time_entry_per_day
        for i in range(0, number_of_time_entry):
            if (i != 0) and (i % number_of_time_entry_per_day == 0):
                date_ts += 86400
                output[user][date_ts] = [-1] * number_of_time_entry_per_day
            output[user][date_ts][i % number_of_time_entry_per_day] = result[user][i]
                
    return result

output = create_time_slot_data(df, {}, 1519376400, 1519545599, 60)
print(output)

24
{12: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1], 79: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1], 81: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1], 50: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 2, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1], 88: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 17: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [44]:
# helper function to lookup a user map location 
# on a given time
def lookup_map_location_by_time(data, client_username, input_time, start_time, window_in_minutes):
    time_ts = convert_time_to_timestamp(input_time)
    start_ts = convert_time_to_timestamp(start_time)
    index = convert_timestamp_to_index(time_ts, start_ts, window_in_minutes)
    return data[client_username][index]

print(lookup_map_location_by_time(output, 133, 'Sat Feb 24 01:20:38 PST 2018',
                                 'Sat Feb 23 01:00:00 PST 2018', 60))

75
